In [18]:
print("Name: SUKRUTHA D")
print("SRN: PES2UG23CS622")

Name: SUKRUTHA D
SRN: PES2UG23CS622


In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 10.3 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv

load_dotenv()

import getpass #For getting API key
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your API key: ")

Enter your API key: ··········


In [3]:
#temperature=0.0 - more redundant answers - same response again and again
#temperature=1.0 - more unique answers

from langchain_google_genai import ChatGoogleGenerativeAI
#model A: The "Accountant" (precision)
llm_focused =  ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0.0)

#Model B: The "Poet" (creativity)
llm_creative =  ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=1.0)

In [4]:
prompt = "Define the word 'Idea' in one sentence."
print("----FOCUSED (temp=0)----")
print(f"Run 1: {llm_focused.invoke(prompt).content}")
print(f"Run 2: {llm_focused.invoke(prompt).content}")

----FOCUSED (temp=0)----
Run 1: An idea is a thought, concept, or suggestion that is formed or exists in the mind.
Run 2: An idea is a thought, concept, or mental image formed in the mind.


In [5]:
print("----Creative (temp=1)---")
print(f"Run 1: {llm_creative.invoke(prompt).content}")
print(f"Run 2: {llm_creative.invoke(prompt).content}")


----Creative (temp=1)---
Run 1: An idea is a mental construct, thought, or concept that exists in the mind, often serving as a plan, inspiration, or understanding.
Run 2: An idea is a thought, concept, or plan that exists in the mind as a result of mental activity or perception.


**Part 1b: Prompts & Parsers**

In [6]:

from dotenv import load_dotenv
load_dotenv()
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter you google API key: ")

llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [7]:
#system message = controls tone + behavior
from langchain_core.messages import SystemMessage,HumanMessage

#Scenario : Make the AI rude.
messages=[
    SystemMessage(content="You are not very good. You are not giving proper answers and you are careless."),
    HumanMessage(content="What is capital of India?")
]

response = llm.invoke(messages)
print(response.content)

The capital of India is New Delhi.


In [10]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate {input_language} to {output_language}."),
    ("human", "{text}")
])
print(f"Required variables: {template.input_variables}")

Required variables: ['input_language', 'output_language', 'text']


In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

raw_msg = llm.invoke("Hi")
print(f"Raw Type: {type(raw_msg)}")


clean_text = parser.invoke(raw_msg)
print(f"Parsed Type: {type(clean_text)}")
print(f"Content: {clean_text}")


Raw Type: <class 'langchain_core.messages.ai.AIMessage'>
Parsed Type: <class 'langchain_core.messages.base.TextAccessor'>
Content: Hello! How can I help you today?


**Part 1c: LCEL (LangChain Expression Language)**

In [12]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter you google API key: ")

llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash")
template = ChatPromptTemplate.from_template("Tell me a fun fact about {topic}");
parser=StrOutputParser()

In [13]:
#Method A: the manual way
prompt_value=template.invoke({"topic": "Gorillas"})

response_obj=llm.invoke(prompt_value)

final_text=parser.invoke(response_obj)
print(final_text)

Here's a fun fact about gorillas:

Gorillas are expert nest-builders! Every evening, they construct a brand new, comfy bed for themselves, either on the ground or up in a tree. They'll use branches, leaves, and twigs to create a safe and cozy spot for the night, essentially getting a fresh bedroom every single day!


In [14]:
chain=template | llm | parser
print(chain.invoke({"topic":"Octopuses"}))

Here's a fun one:

Did you know octopuses have **three hearts**?

*   Two hearts pump blood through their gills.
*   The third, larger heart circulates blood to the rest of their body.

This is because their blood is copper-based (which makes it blue!) and is less efficient at carrying oxygen, so they need extra pumping power!


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chain = ChatPromptTemplate.from_template(
    "Movie: {movie}\nWhat year was it released? Then calculate how many years ago that was from 2026."
) | llm | StrOutputParser()


In [17]:
response = chain.invoke({"movie": "Cruella"})
print(response)


Cruella was released in **2021**.

From 2026, that was **5 years ago** (2026 - 2021 = 5).
